In [3]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
from pyquil.paulis import *
import numpy as np
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize

In [4]:
qvm = api.QVMConnection()
vqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'})

In [30]:
hamiltonian = sX(0)*sZ(1)*sX(2) + sY(0)*sZ(1)*sY(2) + sX(1)*sZ(2)*sX(3) + sY(1)*sZ(2)*sY(3) 

In [31]:
vqe_inst.expectation(test_ansatz(), hamiltonian, None, qvm)

-3.9999999999999982

In [28]:
def test_ansatz():
    p = Program()

    p.inst(H(0))
    p.inst(CNOT(0, 1))
    p.inst(H(0))
    p.inst(CNOT(1, 2))
    p.inst(CNOT(0, 1))
    p.inst(CNOT(2, 3))
    p.inst(CNOT(1, 2))
    p.inst(X(1))
    p.inst(CNOT(2, 3))
    p.inst(Y(2))
    return p

In [46]:
def ansatz(params):
    S = 2
    p = Program()
    p += X(0)          
    p += X(2)
    for i in range(S):

        # building Uu
        for j in range(4):
            p += RZ(params[0]/2, j)
        p += CZ(0,1)
        p += CZ(2,3)
        p += RZ(-params[0]/2, 1)
        p += RZ(-params[0]/2, 3)
        p += CZ(0,1)
        p += CZ(2,3)
        
        # building Ut
        p += CZ(2,3)
        p += CZ(1,2)
        for j in range(4):
            p += H(j)
        p += CNOT(1,3)
        p += CNOT(0,2)
        p += RZ(params[1], 2)
        p += RZ(params[1], 3)
        p += CNOT(1,3)
        p += CNOT(0,2)
        for j in range(4):
            p += H(j)
        for j in range(4):
            p += Y(j)
        p += CNOT(0,2)
        p += CNOT(1,3)
        p += RZ(params[1], 2)
        p += RZ(params[1], 3)
        p += CNOT(0,2)
        p += CNOT(1,3)
        for j in range(4):
            p += Y(j)
        p += CZ(1,2)
        p += CZ(2,3)
    
    return p

In [47]:
#initial angle
initial_params = [0.0, 0.0]
result = vqe_inst.vqe_run(ansatz, hamiltonian, initial_params, None, qvm=qvm)

                     models will be ineffective


In [43]:
print(result)

{'x': array([ 7.8125e-05, -3.1250e-05]), 'fun': -4.235164733282458e-25}
